# 将nl转化为指令微调的格式

In [1]:
from typing import List
from typing import Optional
from typing import Dict

import os
import json
from pathlib import Path

import httpx
from openai import OpenAI

import requests

import time
from tqdm import tqdm
import re

In [2]:
# 文件路径
file_path = "prompts/DataFountain/instruction.txt"

# 请求的 URL
url = "https://api.moonshot.cn/v1/files"

headers = {
    'Authorization': 'sk-gNToXJPO4Ih8NDZnDloSXR780dQDEZ2Epu49wcVA1sWb4Ecw'
}

# 准备文件数据
files = {'file': open(file_path, 'rb')}

# 发起 POST 请求
response = requests.post(url, headers=headers, files=files)

# 打印响应内容
print(response.status_code)
print(response.text)

200
{"id":"ctcl9n3jfih8i2n6966g","object":"file","bytes":3641,"created_at":1733907676,"filename":"instruction.txt","purpose":"file-extract","status":"ok","status_details":""}


In [3]:
client = OpenAI(
    api_key="sk-gNToXJPO4Ih8NDZnDloSXR780dQDEZ2Epu49wcVA1sWb4Ecw",
    base_url="https://api.moonshot.cn/v1",
)

file_object = client.files.create(file=Path("prompts/DataFountain/instruction.txt"), purpose="file-extract")
file_content = client.files.content(file_id=file_object.id).text

## 读取自然语言数据

In [4]:
with open('data/DF/instruction_v1.json', 'r', encoding='utf-8') as f:
    nl_data = json.load(f)

In [5]:
print(nl_data[0])

{'rule_id': 0, 'nl_rule_pair': [{'rule': '危险化学品事故是指危险化学品生产、经营、储存、运输、使用和废弃危险化学品处置等过程中由危险化学品造成人员伤害、财产损失和环境污染的事故（矿山开采过程中发生的有毒、有害气体中毒、爆炸事故、放炮事故除外）。', 'natural_language': '如果工厂发生了爆炸事件，并且涉及的是易燃化学品，那么这个事件会导致人员伤害、财产损失和环境污染。同时，这个事件与生产、操作、储存、运输、使用或处置易燃化学品有关，并且不是矿井事故、有毒气体中毒、爆炸事故或爆破事故。'}, {'rule': '危险化学品事故是指危险化学品生产、经营、储存、运输、使用和废弃危险化学品处置等过程中由危险化学品造成人员伤害、财产损失和环境污染的事故（矿山开采过程中发生的有毒、有害气体中毒、爆炸事故、放炮事故除外）。', 'natural_language': '如果化工厂发生了化学品泄漏事件，并且涉及的是有毒化学品，那么这个事件会导致人员伤害、财产损失和环境污染。同时，这个事件与生产、操作、储存、运输、使用或处置有毒化学品有关，并且不是矿井事故、有毒气体中毒、爆炸事故或爆破事故。'}, {'rule': '危险化学品事故是指危险化学品生产、经营、储存、运输、使用和废弃危险化学品处置等过程中由危险化学品造成人员伤害、财产损失和环境污染的事故（矿山开采过程中发生的有毒、有害气体中毒、爆炸事故、放炮事故除外）。', 'natural_language': '如果在运输途中发生了化学品泄漏事件，并且涉及的是腐蚀性化学品，那么这个事件会导致人员伤害、财产损失和环境污染。同时，这个事件与生产、操作、储存、运输、使用或处置腐蚀性化学品有关，并且不是矿井事故、有毒气体中毒、爆炸事故或爆破事故。'}, {'rule': '危险化学品事故是指危险化学品生产、经营、储存、运输、使用和废弃危险化学品处置等过程中由危险化学品造成人员伤害、财产损失和环境污染的事故（矿山开采过程中发生的有毒、有害气体中毒、爆炸事故、放炮事故除外）。', 'natural_language': '如果实验室发生了化学品爆炸事件，并且涉及的是实验用化学品，那么这个事件会导致人员伤害、财产损失和环境污染。同时，这个事件与生产、操作、储存、运输、使用或处置实验用化学

In [6]:
i = 0


def wrap_with_quotes(inputs):
    # Convert the item to a JSON-formatted string
    json_content = json.dumps(inputs, ensure_ascii=False, indent=4)
    # Wrap the content with triple quotes
    result = f'```json\n{json_content}\n```'
    return result


# Example input: a list of dictionaries
input_data = [
    {
        "rule": "机关服务中心的职责：负责安全监管总局事故应急处置过程中的后勤保障工作。",
        "natural_language": "在事故应急处置过程中，如果任务是事故现场安全检查，那么将提供后勤支持，并且机关服务中心负责这项任务。"
    },
    {
        "rule": "另一个规则示例：说明某些具体职责。",
        "natural_language": "这是另一个自然语言描述示例。"
    }
]

# Process the input data
wrapped_data = wrap_with_quotes(input_data[0])
print(wrapped_data)

```json
{
    "rule": "机关服务中心的职责：负责安全监管总局事故应急处置过程中的后勤保障工作。",
    "natural_language": "在事故应急处置过程中，如果任务是事故现场安全检查，那么将提供后勤支持，并且机关服务中心负责这项任务。"
}
```


## 调用api函数

In [7]:
def call_api_with_rate_limit_handling(client, messages, model="moonshot-v1-32k", max_retries=5):
    retries = 0
    while retries < max_retries:
        try:
            completion = client.chat.completions.create(
                model=model,
                messages=messages,
                temperature=0,
            )
            return completion
        except Exception as e:
            if '429' in str(e):  # 检查异常消息中是否包含429
                print("Rate limit reached. Retrying in 1 second...")
                time.sleep(1)  # 等待1秒钟
                retries += 1
            else:
                raise  # Re-raise the exception if it's not a 429 error
    raise Exception("Max retries reached. API call failed.")

## 调用api

In [8]:
answer = []
for data in tqdm(nl_data, desc="Processing"):
    dicts = {
        "rule_id": data["rule_id"],
        "io_pair": []
    }
    for pair in data["nl_rule_pair"]:
        wrapped_data = wrap_with_quotes(pair)
        # 替换内容
        replacement = wrapped_data

        # 使用正则表达式替换 [[RULES]]
        content = re.sub(r'\[\[RULES\]\]', replacement, file_content)

        messages = [
            {
                "role": "system",
                "content": "你是 Kimi，由 Moonshot AI 提供的人工智能助手，你更擅长中文和英文的对话。你会为用户提供安全，有帮助，准确的回答。同时，你会拒绝一切涉及恐怖主义，种族歧视，黄色暴力等问题的回答。Moonshot AI 为专有名词，不可翻译成其他语言。请将内容输出为可解析的json格式。"
            },
            {
                "role": "system",
                "content": content,
            },
            {
                "role": "user",
                "content": "你好，请根据txt内容生成回答。"
            }
        ]

        completion = call_api_with_rate_limit_handling(client, messages, "moonshot-v1-32k", 5)

        dicts["io_pair"].append(completion.choices[0].message.content)
    answer.append(dicts)

Processing: 100%|██████████| 800/800 [7:46:58<00:00, 35.02s/it]    


In [ ]:
print(answer[0]["io_pair"][0])
print(len(answer))

In [9]:
with open('data/DF/instruction_transed_v1.json', 'w', encoding='utf-8') as f:
    json.dump(answer, f, ensure_ascii=False, indent=4)

# 缓存

In [1]:
from typing import List
from typing import Optional
from typing import Dict

import os
import json
import jsonlines
from pathlib import Path

import httpx
from openai import OpenAI

import requests

import time
import re
from tqdm import tqdm

In [2]:
# 你的API密钥
api_key = "sk-gNToXJPO4Ih8NDZnDloSXR780dQDEZ2Epu49wcVA1sWb4Ecw"

# 读取文本文件内容
with open('prompts/DataFountain/instruction.txt', 'r', encoding='utf-8') as file:
    file_content = file.read()

# 创建缓存的请求数据
data = {
    "model": "moonshot-v1",
    "messages": [
        {
            "role": "system",
            "content": file_content
        }
    ],
    "name": "example_cache",
    "ttl": 3600  # 缓存有效期，单位为秒
}

# 发送POST请求创建缓存
response = requests.post(
    url="https://api.moonshot.cn/v1/caching",
    headers={
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json"
    },
    json=data
)

# 打印响应内容
print(json.loads(response.text))

{'id': 'cache-eztxpkuoc6di11ggm7a1', 'object': 'cache_object', 'model': 'moonshot-v1', 'messages': [{'role': 'system', 'content': '任务描述：你将被给予一段文字，这段文字是由如下步骤生成的：（1）将规则转换为对应逻辑表达式的形式，并尽量确保转换过程不丢失关键信息；（2）由逻辑表达式取不同的值，确保涵盖不同的场景、条件和响应行为生成；（3）将取不同值的逻辑表达式实例化，转化为自然语言的形式。你的任务是将某个规则对应的自然语言，生成对话式训练数据数据，即将自然语言转化为问答式指令微调数据的形式。\n------\n下面是一个例子：\n```json\n{\n    "rule": "安全生产协调司的职责：根据安全监管总局领导指示和有关规定，组织协调安全监察专员赶赴事故现场参与事故应急救援和事故调查处理工作。",\n    "natural_language": "如果在化工厂发生了事故，并且领导根据“化工事故应急处理规定”发出了指示，那么安全监察专员张三会接到指令，立即前往化工厂，参与现场的应急救援工作和事故调查。"\n}\n```\n###\n这条例子转化为问答式指令微调数据的形式如下：\n```json\n{\n    "instruction": "如果在化工厂发生了事故，并且领导根据“化工事故应急处理规定”发出了指示，安全监察专员张三应该做什么？",\n    "output": "安全监察专员张三会接到指令，立即前往化工厂，参与现场的应急救援工作和事故调查。"\n}\n```\n------\n下面是一个例子：\n```json\n{\n    "rule": "调度统计司的职责：负责应急值守，接收、处置各地、各部门上报的事故信息，及时报告安全监管总局领导，同时转送安全监管总局办公厅和应急指挥中心；按照安全监管总局领导指示，起草事故救援处理工作指导意见；跟踪、续报事故救援进展情况。",\n    "natural_language": "如果调度统计司承担紧急值班任务，并且接收到化工厂爆炸事故的信息，并且处理了化工厂爆炸事故的信息，并且向领导报告了化工厂爆炸事故的信息，并且将化工厂爆炸事故的信息转发到办公室，并且将化工厂爆炸事故的信

In [4]:
# 读取文本文件内容
def load_file_content(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read()


# 创建缓存
def create_cache(file_content):
    data = {
        "model": "moonshot-v1",
        "messages": [
            {
                "role": "system",
                "content": file_content
            }
        ],
        "name": "example_cache",
        "ttl": 3600  # 缓存有效期，单位为秒
    }
    response = requests.post(
        url="https://api.moonshot.cn/v1/caching",
        headers={
            "Authorization": f"Bearer {api_key}",
            "Content-Type": "application/json"
        },
        json=data
    )
    cache_response = json.loads(response.text)
    return cache_response['id']


# 检查缓存是否存在且未过期
def check_cache(cache_id):
    response = requests.get(
        url=f"https://api.moonshot.cn/v1/caching/{cache_id}",
        headers={
            "Authorization": f"Bearer {api_key}"
        }
    )
    if response.status_code == 200:
        return True
    return False


# 重新加载数据并更新缓存
def reload_and_update_cache(file_path, cache_id):
    new_file_content = load_file_content(file_path)
    data = {
        "model": "moonshot-v1",
        "messages": [
            {
                "role": "system",
                "content": new_file_content
            }
        ],
        "name": "example_cache",
        "ttl": 3600  # 缓存有效期，单位为秒
    }
    response = requests.put(
        url=f"https://api.moonshot.cn/v1/caching/{cache_id}",
        headers={
            "Authorization": f"Bearer {api_key}",
            "Content-Type": "application/json"
        },
        json=data
    )
    return response.status_code == 200


# 使用缓存内容并添加问题
def use_cache_with_question(cache_id, question):
    data = {
        "model": "moonshot-v1-32k",
        "messages": [
            {
                "role": "cache",
                "content": f"cache_id={cache_id};reset_ttl=3600",
            },
            {
                "role": "user",
                "content": question
            }
        ],
        "max_tokens": 8192,
    }
    response = requests.post(
        url="https://api.moonshot.cn/v1/chat/completions",
        headers={
            "Authorization": f"Bearer {api_key}",
            "Content-Type": "application/json"
        },
        json=data
    )
    return response.json()['choices'][0]['message']['content']

In [5]:
# 主流程
file_path = 'prompts/DataFountain/instruction.txt'
file_content = load_file_content(file_path)
print(f"File{file_path} success read")
cache_id = create_cache(file_content)
print(f"cache id:{cache_id}")
print("cache cuccess")

# # 模拟缓存过期
# time.sleep(3600)

# 检查缓存是否存在且未过期
if not check_cache(cache_id):
    print("检查不到缓存")
    # 重新加载数据并更新缓存
    if reload_and_update_cache(file_path, cache_id):
        print("缓存更新成功")
    else:
        print("缓存更新失败")

print("缓存已存在")

with open('data/RJUA/RJUA_predicates2nl_v3.json', 'r', encoding='utf-8') as file:
    answer = json.load(file)

for idx, value in tqdm(enumerate(answer), total=len(answer), desc="Processing"):
    context = json.dumps({
        "rule": value['rule'],
        "natural_language": value['natural_language']
    }, indent=4, ensure_ascii=False)
    # 使用缓存内容并添加问题
    question = f"请回答关于文件内容的问题，其中[[CONTEXT]]代表的数据为{context}"
    response = use_cache_with_question(cache_id, question)
    judgement = {
        "id": value["id"],
        "number": value['number'],
        "rule": value['rule'],
        "nl": value['natural_language'],
        "instruction": response
    }
    # 打开文件以进行写入，如果文件不存在，会创建文件
    with jsonlines.open('data/RJUA/RJUA_nl2instruction_v1.jsonl', mode='a') as writer:
        writer.write(judgement)


Fileprompts/DataFountain/instruction.txt success read
cache id:cache-eztxt35oc6di11ggm8c1
cache cuccess
缓存已存在


Processing: 100%|██████████| 1772/1772 [1:17:50<00:00,  2.64s/it]  
